In [1]:
!pip install -e /home/lars/devel/tpk4170-robotics/

Obtaining file:///home/lars/devel/tpk4170-robotics
  Found existing installation: tpk4170 1.0
    Uninstalling tpk4170-1.0:
      Successfully uninstalled tpk4170-1.0
  Running setup.py develop for tpk4170
You are using pip version 10.0.1, however version 18.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [17]:
import numpy as np
from tpk4170.models import ColladaMesh, Grid, Axes
from tpk4170.visualization import Viewer
from tpk4170.utils.transformations import quaternion_from_euler

In [52]:
base_link = ColladaMesh('./robotiq/2f85/robotiq_85_base_link.dae')
right_finger_link = ColladaMesh('./robotiq/2f85/robotiq_85_finger_link.dae')
left_finger_link = ColladaMesh('./robotiq/2f85/robotiq_85_finger_link.dae')
finger_tip_link = ColladaMesh('./robotiq/2f85/robotiq_85_finger_tip_link.dae')
left_inner_knuckle_link = ColladaMesh('./robotiq/2f85/robotiq_85_inner_knuckle_link.dae')
right_inner_knuckle_link = ColladaMesh('./robotiq/2f85/robotiq_85_inner_knuckle_link.dae')
left_knuckle_link = ColladaMesh('./robotiq/2f85/robotiq_85_knuckle_link.dae')
right_knuckle_link = ColladaMesh('./robotiq/2f85/robotiq_85_knuckle_link.dae')

In [53]:
viewer = Viewer(background='white', camera_position=[1,1,1])
viewer.add(Grid())

Renderer(camera=PerspectiveCamera(aspect=1.5, children=(DirectionalLight(color='white', intensity=0.66, positi…

In [54]:
viewer.add(base_link)

In [55]:
left_knuckle_link.quaternion = quaternion_from_euler(np.pi,0.0,0.0).tolist()
left_knuckle_link.position = [0.05490451627, 0.03060114443, 0.0]
base_link.add(left_knuckle_link)

In [58]:
# right_knuckle_link.quaternion = quaternion_from_euler(0,0.0,0.0).tolist()
right_knuckle_link.position = [0.05490451627, -0.03060114443, 0.0]
base_link.add(right_knuckle_link)

In [59]:
left_finger_link.position = [-0.00408552455, -0.03148604435, 0.0]
left_knuckle_link.add(left_finger_link)

In [57]:
right_finger_link.position = [-0.00408552455, -0.03148604435, 0.0]
right_knuckle_link.add(right_finger_link)